In [1]:
# coding:utf-8
import pandas as pd
import numpy as np
from scipy import sparse
import gc
from tqdm import tqdm
import time
import warnings
import lightgbm as lgb
import os

warnings.filterwarnings('ignore')

D:\Program Files\Anaconda\lib\site-packages\dask\dataframe\utils.py:367: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
D:\Program Files\Anaconda\lib\site-packages\dask\dataframe\utils.py:367: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
D:\Program Files\Anaconda\lib\site-packages\dask\dataframe\utils.py:367: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


ModuleNotFoundError: No module named 'modin'

In [ ]:
from pandarallel import pandarallel
# pandarallel will determine how many cores you have, but you can specify it yourself
pandarallel.initialize(progress_bar=False)

In [ ]:
path = '..'

In [ ]:
date_list = pd.date_range('2014-12-13', '2014-12-18')
date_list = [str(x).split(' ')[0] for x in date_list]

In [ ]:
for date_ in reversed(date_list):
    print(date_)

In [ ]:
# user_id item_id behavior_type user_geohash item_category time
# item_id item_category item_geohash

In [ ]:
# # 大数据版本
# for data_index in [0,1,2,3,4]:
#     t = mpd.read_csv(path + '/data/round1_user_{}.txt'.format(data_index),sep='\t',header=None)
#     t['date'] = t[5].apply(lambda x: str(x).split(' ')[0])
#     for date_ in date_list:
#         print(date_)
#         tt = t[t['date']==date_]
#         print(tt['date'].unique(),date_,tt.shape)
#         tt.columns = [str(x) for x in tt.columns]
#         tt.to_parquet(path + "/sub_data/by_date/round1_user_{}_{}.parquet".format(date_, data_index))
#     del t
#     gc.collect()

In [ ]:
for date_ in reversed(date_list):
    print('begin',date_)
    for data_index in [0, 1, 2, 3, 4]:
        print(path + '/data/round1_user_{}.txt'.format(data_index))
        tmp_round_user = pd.read_csv(path + '/data/round1_user_{}.txt'.format(data_index),
                                     sep='\t',
                                     header=None,
                                     chunksize=5000000
                                     )
        parquet_data = []
        for t in tmp_round_user:
            t['date'] = t[5].apply(lambda x: str(x).split(' ')[0])
            t = t[t['date'] == date_]
            parquet_data.append(t)
        parquet_data = pd.concat(parquet_data,copy=False)
        parquet_data.columns = [str(x) for x in parquet_data.columns]
        parquet_data.to_parquet(path + "/sub_data/by_date/round1_user_{}_{}.parquet".format(date_, data_index))
        print(parquet_data.shape)
        print(parquet_data['date'].unique())
        del parquet_data,t
    del tmp_round_user
    gc.collect()

In [ ]:
# 1.读取12-17数据，保留12-17行为为4的数据，保存为train-label 
data_date = '2014-12-17'
print('for train data set')
train_label = []
for data_index in [0, 1, 2, 3, 4]:
    tmp_parquet = pd.read_parquet(path + "/sub_data/by_date/round1_user_{}_{}.parquet".format(data_date,data_index))
#   user_id item_id behavior_type user_geohash item_category time
    tmp_parquet = tmp_parquet[tmp_parquet['2'] == 4][['0', '1']]
    print(tmp_parquet.shape)
    train_label.append(tmp_parquet)
    del tmp_parquet
train_label = pd.concat(train_label,copy=False).drop_duplicates()
train_label['label'] = 1
train_label.columns = ['user_id', 'item_id', 'label']
train_label.to_parquet(path + "/sub_data/use_data/train_label.parquet")
print(train_label.shape)
del train_label
gc.collect()

# 2.构造12-17日所有item item_category 构造候选数据集
train_item = []
for data_index in [0, 1, 2, 3, 4]:
    tmp_parquet = pd.read_parquet(path + "/sub_data/by_date/round1_user_{}_{}.parquet".format(data_date,data_index))
#   item_id item_category
    tmp_parquet = tmp_parquet[['1', '4']]
    print(tmp_parquet.shape)
    train_item.append(tmp_parquet)
    del tmp_parquet
train_item = pd.concat(train_item,copy=False).drop_duplicates()
train_item.columns = ['item_id', 'item_category']
train_item.to_parquet(path + "/sub_data/use_data/train_item.parquet")
print(train_item.shape)
del train_item
gc.collect()

In [ ]:
# 1.读取12-18数据，保留12-18行为为4的数据，保存为valid-label,去重
data_date = '2014-12-18'
print('for valid data set')
valid_label = []
for data_index in [0, 1, 2, 3, 4]:
    tmp_parquet = pd.read_parquet(path + "/sub_data/by_date/round1_user_{}_{}.parquet".format(data_date,data_index))
    tmp_parquet = tmp_parquet[tmp_parquet['2'] == 4][['0', '1']]
    print(tmp_parquet.shape)
    valid_label.append(tmp_parquet)
    del tmp_parquet
valid_label = pd.concat(valid_label).drop_duplicates()
valid_label['label'] = 1
valid_label.columns = ['user_id', 'item_id', 'label']
valid_label.to_parquet(path + "/sub_data/use_data/valid_label.parquet")
print(valid_label.shape)
del valid_label
gc.collect()

# 2.构造12-18日所有item item_category 构造候选数据集
valid_item = []
for data_index in [0, 1, 2, 3, 4]:
    tmp_parquet = pd.read_parquet(path + "/sub_data/by_date/round1_user_{}_{}.parquet".format(data_date,data_index))
    tmp_parquet = tmp_parquet[['1', '4']]
    print(tmp_parquet.shape)
    valid_item.append(tmp_parquet)
    del tmp_parquet
valid_item = pd.concat(valid_item).drop_duplicates()
valid_item.columns = ['item_id', 'item_category']
valid_item.to_parquet(path + "/sub_data/use_data/valid_item.parquet")
print(valid_item.shape)
del valid_item
gc.collect()

In [ ]:
def get_history_data(label,item,user_data_list,result_name,is_not_submit):
    feature = ['user_id', 'item_id', 'behavior_type', 'user_geohash', 'item_category', 'time']
    # 读取历史区间的数据
    data = []
    # user_data_list = {'2014-12-16': 0, '2014-12-15': 1}
    for use_date in user_data_list:
        for data_index in [0, 1, 2, 3, 4]:
            tmp_parquet = pd.read_parquet(path + "/sub_data/by_date/round1_user_{}_{}.parquet".format(use_date, data_index))
            del tmp_parquet['date']
            tmp_parquet.columns = feature
            print(tmp_parquet.shape)
            tmp_parquet = tmp_parquet[tmp_parquet['item_id'].isin(item['item_id'].unique())]
            print(tmp_parquet.shape)
            tmp_parquet['gap'] = tmp_parquet['time'].apply(lambda x: str(x).split(' ')[0])
            tmp_parquet['hour'] = tmp_parquet['time'].apply(lambda x: int(str(x).split(' ')[1]))
            tmp_parquet['gap'] = tmp_parquet['gap'].map(user_data_list)
            del tmp_parquet['user_geohash']
            del tmp_parquet['time']
            data.append(tmp_parquet)
            del tmp_parquet
            gc.collect()
    data = pd.concat(data,copy=False).reset_index(drop=True)
    print(data.shape)
    data.to_parquet(path + "/sub_data/use_data/history_{}.parquet".format(result_name))
    return data

In [ ]:
def my_ptp(x):
    return np.max(x) - np.min(x)

In [ ]:
# 选择gap区间，统计以key1为主键，key2为条件，是否全部行为bh内的 func特征
#  get_windows_feat(data,1,'user_item','f',[1,2,3,4],['count'],False)
#  get_windows_feat(data,1,'user_item','offset',[1,2,3,4],['min','max','nunique','var'],False)
#  get_windows_feat(data,1,'user_item','gap',[1,2,3,4],['nunique'],True)
def get_windows_feat(data,gap,key1,key2,bh,fnc,is_eq):
    # 构造逆序时间信息
    if key2 == 'offset':
        data['offset'] = data['gap'] * 24 + 24 - data['hour']
    if key2 == 'f':
        data['f'] = 1
    if is_eq:
        tt = data[(data['gap']==gap)&(data['behavior_type'].isin(bh))].groupby(key1).agg({key2:fnc}).reset_index()
        tt.columns = ['w_eq{}b{}_{}'.format(gap,''.join(map(str,bh)),key1) + x[0] +'_'+ x[1] if x[1]!='' else x[0] for x in tt.columns]
    else:
        tt = data[(data['gap']<=gap)&(data['behavior_type'].isin(bh))].groupby(key1).agg({key2:fnc}).reset_index() 
        tt.columns = ['w_lt{}b{}_{}'.format(gap,''.join(map(str,bh)),key1) + x[0] +'_'+ x[1] if x[1]!='' else x[0] for x in tt.columns]
    tt = tt.fillna(0)
#     tt = reduce_mem_usage(tt)
    print(gap,tt.shape)
    return tt

In [ ]:
# 主要针对behavior_type信息做展开特征
#  get_pivot_table_feature(data,1,'user_item','behavior_type',False)
#  get_pivot_table_feature(data,1,'user_item','behavior_type',True)
def get_pivot_table_feature(data,gap,key1,key2,is_eq):
    def get_ctr(x):
        x = x.values
#         print(x)
        res = [x[3]/(1+x[0]),x[3]/(1+x[1]),x[3]/(1+x[2]),x[2]/(1+x[0]),x[1]/(1+x[0])]
        if x[0] == 0:
            res[0] = -1
            res[3] = -1
            res[4] = -1
        if x[1] == 0:
            res[1] = -1
        if x[2] == 0:
            res[2] = -1
        return res

    if is_eq:
        t = data[data['gap']==gap]
    else:
        t = data[data['gap']<=gap]
    t['f'] = 1
    t = pd.pivot_table(t,'f',key1,key2,aggfunc='sum', fill_value=0).reset_index()
    # 多进程加速
    t[['4_1','4_2','4_3','2_1','3_1']] = t[[1,2,3,4]].parallel_apply(lambda x:get_ctr(x),axis=1,result_type="expand")
    if is_eq:    
        t.columns = ['weq{}ctr'.format(gap)+ '_' +key1 +'_' +key2 +'_' + str(x) if x != key1 else key1 for x in t.columns]
    else:
        t.columns = ['wlt{}ctr'.format(gap)+ '_' +key1 +'_' +key2 +'_' + str(x) if x != key1 else key1 for x in t.columns]       
    t = t.fillna(0)
#     t = reduce_mem_usage(t)
    print(gap,t.shape)
    return t

In [ ]:
def gen_feature(data, item, label, is_submit, name):
    data['user_category'] = data[['user_id','item_category']].parallel_apply(lambda x:str(x[0]) +'_' + str(x[1]),axis=1)
#     data['item_hour'] = data[['item_id','hour']].parallel_apply(lambda x:str(x[0]) +'_' + str(x[1]),axis=1)
#     data['user_item_hour'] = data[['user_item','hour']].parallel_apply(lambda x:str(x[0]) +'_' + str(x[1]),axis=1)

        
#     base_data = pd.concat([data[data['gap']==0],data[(data['gap']!=0)&(data['behavior_type']!=1)]],copy=False)
#     print(data.shape)
    base_data = data[data['item_id'].isin(list(item['item_id'].unique()))]
    base_data = base_data[['user_id','item_id','item_category','user_item','user_category']].drop_duplicates()
    print(base_data.shape)
    print(data.head())
    
    print(data.shape)
    data = data[data['item_id'].isin(list(item['item_id'].unique()))].reset_index(drop=True)
    print(data.shape)
    
    print('========== user ==========')
#  get_windows_feat(data,1,'user_item','f',[1,2,3,4],['count'],False)
#  get_windows_feat(data,1,'user_item','offset',[1,2,3,4],['min','max','nunique','var'],False)
#  get_windows_feat(data,1,'user_item','gap',[1,2,3,4],['nunique'],True)
# 统计用户交互过的商品个数和数量 
    for windows in [0]:
        print('get_windows_feat user_id item_id 1,2,3,4 nunique count {} False'.format(windows))
        if os.path.exists(path + "/sub_data/use_data/{}_u_1_w_{}.parquet".format(name,windows)):
            t = pd.read_parquet(path + "/sub_data/use_data/{}_u_1_w_{}.parquet".format(name,windows))
        else:
            t = get_windows_feat(data,windows,'user_id','item_id',[1,2,3,4],['nunique','count'],False)
            print(t.head())
            t.to_parquet(path + "/sub_data/use_data/{}_u_1_w_{}.parquet".format(name,windows))
        base_data = pd.merge(base_data,t,on=['user_id'],how='left',copy=False)
        del t;gc.collect()
        
    for windows in [0]:
        print('get_windows_feat user_id item_id 1,2,3,4 nunique count {} False'.format(windows))
        if os.path.exists(path + "/sub_data/use_data/{}_u_104_w_{}.parquet".format(name,windows)):
            t = pd.read_parquet(path + "/sub_data/use_data/{}_u_104_w_{}.parquet".format(name,windows))
        else:
            t = get_windows_feat(data,windows,'user_id','item_id',[3,4],['nunique','count'],False)
            print(t.head())
            t.to_parquet(path + "/sub_data/use_data/{}_u_104_w_{}.parquet".format(name,windows))
        base_data = pd.merge(base_data,t,on=['user_id'],how='left',copy=False)
        del t;gc.collect()  
        
# 统计用户交互过的商品个数和数量 
    for windows in [0]:
        print('get_windows_feat user_id item_id 4 nunique count {} False'.format(windows))
        if os.path.exists(path + "/sub_data/use_data/{}_u_101_w_{}.parquet".format(name,windows)):
            t = pd.read_parquet(path + "/sub_data/use_data/{}_u_101_w_{}.parquet".format(name,windows))
        else:
            t = get_windows_feat(data,windows,'user_id','item_id',[4],['nunique','count'],False)
            print(t.head())
            t.to_parquet(path + "/sub_data/use_data/{}_u_101_w_{}.parquet".format(name,windows))
        base_data = pd.merge(base_data,t,on=['user_id'],how='left',copy=False)
        del t;gc.collect()
        
# 统计用户交互过的商品类型个数和数量 
    for windows in [0]:
        print('get_windows_feat user_id item_category 1,2,3,4 nunique {} False'.format(windows))
        if os.path.exists(path + "/sub_data/use_data/{}_u_201_w_{}.parquet".format(name,windows)):
            t = pd.read_parquet(path + "/sub_data/use_data/{}_u_201_w_{}.parquet".format(name,windows))
        else:
            t = get_windows_feat(data,windows,'user_id','item_category',[1,2,3,4],['count','nunique'],False)
            print(t.head())
            t.to_parquet(path + "/sub_data/use_data/{}_u_201_w_{}.parquet".format(name,windows))
        base_data = pd.merge(base_data,t,on=['user_id'],how='left',copy=False)
        del t;gc.collect()

    for windows in [0]:
        print('get_windows_feat user_id item_category 1,2,3,4 nunique {} False'.format(windows))
        if os.path.exists(path + "/sub_data/use_data/{}_u_204_w_{}.parquet".format(name,windows)):
            t = pd.read_parquet(path + "/sub_data/use_data/{}_u_204_w_{}.parquet".format(name,windows))
        else:
            t = get_windows_feat(data,windows,'user_id','item_category',[4],['count','nunique'],False)
            print(t.head())
            t.to_parquet(path + "/sub_data/use_data/{}_u_204_w_{}.parquet".format(name,windows))
        base_data = pd.merge(base_data,t,on=['user_id'],how='left',copy=False)
        del t;gc.collect()        
        
# 统计用户不同行为的频次
#  get_pivot_table_feature(data,1,'user_item','behavior_type',False)
#  get_pivot_table_feature(data,1,'user_item','behavior_type',True)
    for windows in [0]:
        print('get_pivot_table_feature user_id behavior_type {} False'.format(windows))
        if os.path.exists(path + "/sub_data/use_data/{}_u_3_w_{}.parquet".format(name,windows)):
            t = pd.read_parquet(path + "/sub_data/use_data/{}_u_3_w_{}.parquet".format(name,windows))
        else:
            t = get_pivot_table_feature(data,windows,'user_id','behavior_type',False)
            print(t.head())
            t.to_parquet(path + "/sub_data/use_data/{}_u_3_w_{}.parquet".format(name,windows))
        base_data = pd.merge(base_data,t,on=['user_id'],how='left',copy=False)
        del t;gc.collect()

    for windows in [0]:
        print('get_pivot_table_feature user_id behavior_type {} False'.format(windows))
        if os.path.exists(path + "/sub_data/use_data/{}_u_303_w_{}.parquet".format(name,windows)):
            t = pd.read_parquet(path + "/sub_data/use_data/{}_u_303_w_{}.parquet".format(name,windows))
        else:
            t = get_pivot_table_feature(data,windows,'user_id','hour',False)
            print(t.head())
            t.to_parquet(path + "/sub_data/use_data/{}_u_303_w_{}.parquet".format(name,windows))
        base_data = pd.merge(base_data,t,on=['user_id'],how='left',copy=False)
        del t;gc.collect()

# 统计用户在重构时间轴内的连续值特征
#      get_windows_feat(data,1,'user_item','offset',[1,2,3,4],['min','max','nunique','var'],False)
    for windows in [0]:
        print('get_windows_feat user_id offset 1,2,3,4 max min {} False'.format(windows))
        if os.path.exists(path + "/sub_data/use_data/{}_u_401_w_{}.parquet".format(name,windows)):
            t = pd.read_parquet(path + "/sub_data/use_data/{}_u_401_w_{}.parquet".format(name,windows))
        else:
            t = get_windows_feat(data,windows,'user_id','hour',[1,2,3,4],['max','min','mean','var'],False)
            print(t.head())
            t.to_parquet(path + "/sub_data/use_data/{}_u_401_w_{}.parquet".format(name,windows))
        base_data = pd.merge(base_data,t,on=['user_id'],how='left',copy=False)
        del t;gc.collect()
    print(base_data.shape)
#      get_windows_feat(data,1,'user_item','offset',[1,2,3,4],['min','max','nunique','var'],False)
    for windows in [0]:
        print('get_windows_feat user_id offset 1,2,3,4 max min {} False'.format(windows))
        if os.path.exists(path + "/sub_data/use_data/{}_u_5_w_{}.parquet".format(name,windows)):
            t = pd.read_parquet(path + "/sub_data/use_data/{}_u_5_w_{}.parquet".format(name,windows))
        else:
            t = get_windows_feat(data,windows,'user_id','offset',[4],['max','min'],False)
            print(t.head())
            t.to_parquet(path + "/sub_data/use_data/{}_u_5_w_{}.parquet".format(name,windows))
        base_data = pd.merge(base_data,t,on=['user_id'],how='left',copy=False)
        del t;gc.collect()
    print(base_data.shape)
    
    for windows in [0]:
        print('get_windows_feat user_id offset 1,2,3,4 max min {} False'.format(windows))
        if os.path.exists(path + "/sub_data/use_data/{}_u_6_w_{}.parquet".format(name,windows)):
            t = pd.read_parquet(path + "/sub_data/use_data/{}_u_6_w_{}.parquet".format(name,windows))
        else:
            t = get_windows_feat(data,windows,'user_id','hour',[4],['max','min'],False)
            print(t.head())
            t.to_parquet(path + "/sub_data/use_data/{}_u_6_w_{}.parquet".format(name,windows))
        base_data = pd.merge(base_data,t,on=['user_id'],how='left',copy=False)
        del t;gc.collect()
    print(base_data.shape)    
    print('========== item ==========')
# 统计商品交互过多少用户和频次
    for windows in [0]:
        print('get_windows_feat item_id user_id 1,2,3,4 nunique count {} False'.format(windows))
        if os.path.exists(path + "/sub_data/use_data/{}_i_1_w_{}.parquet".format(name,windows)):
            t = pd.read_parquet(path + "/sub_data/use_data/{}_i_1_w_{}.parquet".format(name,windows))
        else:
            t = get_windows_feat(data,windows,'item_id','user_id',[1,2,3,4],['nunique','count'],False)
            print(t.head())
            t.to_parquet(path + "/sub_data/use_data/{}_i_1_w_{}.parquet".format(name,windows))
        base_data = pd.merge(base_data,t,on=['item_id'],how='left',copy=False)
        del t;gc.collect()

# 统计商品交互过多少用户和频次
    for windows in [0]:
        print('get_windows_feat item_id user_id 1,2,3,4 nunique count {} False'.format(windows))
        if os.path.exists(path + "/sub_data/use_data/{}_i_104_w_{}.parquet".format(name,windows)):
            t = pd.read_parquet(path + "/sub_data/use_data/{}_i_104_w_{}.parquet".format(name,windows))
        else:
            t = get_windows_feat(data,windows,'item_id','user_id',[4],['nunique','count'],False)
            print(t.head())
            t.to_parquet(path + "/sub_data/use_data/{}_i_104_w_{}.parquet".format(name,windows))
        base_data = pd.merge(base_data,t,on=['item_id'],how='left',copy=False)
        del t;gc.collect()
        
# 统计商品被购买点击加购物车的行为特征
    for windows in [0]:
        print('get_pivot_table_feature item_id behavior_type {} False'.format(windows))
        if os.path.exists(path + "/sub_data/use_data/{}_i_2_w_{}.parquet".format(name,windows)):
            t = pd.read_parquet(path + "/sub_data/use_data/{}_i_2_w_{}.parquet".format(name,windows))
        else:
            t = get_pivot_table_feature(data,windows,'item_id','behavior_type',False)
            print(t.head())
            t.to_parquet(path + "/sub_data/use_data/{}_i_2_w_{}.parquet".format(name,windows))
        base_data = pd.merge(base_data,t,on=['item_id'],how='left',copy=False)
        del t;gc.collect()
    print(base_data.shape)
    
    for windows in [0]:
        print('get_pivot_table_feature item_id behavior_type {} False'.format(windows))
        if os.path.exists(path + "/sub_data/use_data/{}_i_3_w_{}.parquet".format(name,windows)):
            t = pd.read_parquet(path + "/sub_data/use_data/{}_i_3_w_{}.parquet".format(name,windows))
        else:
            t = get_pivot_table_feature(data,windows,'item_id','hour',False)
            print(t.head())
            t.to_parquet(path + "/sub_data/use_data/{}_i_3_w_{}.parquet".format(name,windows))
        base_data = pd.merge(base_data,t,on=['item_id'],how='left',copy=False)
        del t;gc.collect()
    print(base_data.shape)    
    print('========== user item ==========')
# 过去n天 用户-商品对 出现的统计特征 统计用户+商品的统计特征

    for windows in [0]:
        print('get_windows_feat  user_item f 1,2,3,4 count {} False'.format(windows))
        if os.path.exists(path + "/sub_data/use_data/{}_ui_1_w_{}.parquet".format(name,windows)):
            t = pd.read_parquet(path + "/sub_data/use_data/{}_ui_1_w_{}.parquet".format(name,windows))
        else:
            t = get_windows_feat(data,windows,'user_item','f',[1,2,3,4],['count'],False)
            print(t.head())
            t.to_parquet(path + "/sub_data/use_data/{}_ui_1_w_{}.parquet".format(name,windows))
        base_data = pd.merge(base_data,t,on=['user_item'],how='left',copy=False)
        del t;gc.collect()  
        
# 过去N天，user-item 的行为特征的展开表达
    for windows in [0]:
        print('get_pivot_table_feature user_item behavior_type {} False'.format(windows))
        if os.path.exists(path + "/sub_data/use_data/{}_ui_2_w_{}.parquet".format(name,windows)):
            t = pd.read_parquet(path + "/sub_data/use_data/{}_ui_2_w_{}.parquet".format(name,windows))
        else:
            t = get_pivot_table_feature(data,windows,'user_item','behavior_type',False)
            print(t.head())
            t.to_parquet(path + "/sub_data/use_data/{}_ui_2_w_{}.parquet".format(name,windows))
        base_data = pd.merge(base_data,t,on=['user_item'],how='left',copy=False)
        del t;gc.collect()

    for windows in [0]:
        print('get_pivot_table_feature user_item_hour behavior_type {} False'.format(windows))
        if os.path.exists(path + "/sub_data/use_data/{}_ui_202_w_{}.parquet".format(name,windows)):
            t = pd.read_parquet(path + "/sub_data/use_data/{}_ui_202_w_{}.parquet".format(name,windows))
        else:
            t = get_pivot_table_feature(data,windows,'user_item','hour',False)
            print(t.head())
            t.to_parquet(path + "/sub_data/use_data/{}_ui_202_w_{}.parquet".format(name,windows))
        base_data = pd.merge(base_data,t,on=['user_item'],how='left',copy=False)
        del t;gc.collect()
        
# user-item 最近三天内，用户与对应商品最近一次，最远一次出现的 gap        
    for windows in [0]:
        print('get_windows_feat user_item gap 1,2,3,4 max min {} False'.format(windows))
        if os.path.exists(path + "/sub_data/use_data/{}_ui_3_w_{}.parquet".format(name,windows)):
            t = pd.read_parquet(path + "/sub_data/use_data/{}_ui_3_w_{}.parquet".format(name,windows))
        else:
            t = get_windows_feat(data,windows,'user_item','offset',[1,2,3,4],['max','min','nunique'],False)
            print(t.head())
            t.to_parquet(path + "/sub_data/use_data/{}_ui_3_w_{}.parquet".format(name,windows))
        base_data = pd.merge(base_data,t,on=['user_item'],how='left',copy=False)
        del t;gc.collect()
        
    for windows in [0]:
        print('get_windows_feat user_item gap 1,2,3,4 max min {} False'.format(windows))
        if os.path.exists(path + "/sub_data/use_data/{}_ui_4_w_{}.parquet".format(name,windows)):
            t = pd.read_parquet(path + "/sub_data/use_data/{}_ui_4_w_{}.parquet".format(name,windows))
        else:
            t = get_windows_feat(data,windows,'user_item','hour',[1,2,3,4],['max','min'],False)
            print(t.head())
            t.to_parquet(path + "/sub_data/use_data/{}_ui_4_w_{}.parquet".format(name,windows))
        base_data = pd.merge(base_data,t,on=['user_item'],how='left',copy=False)
        del t;gc.collect()
# uc 特征        
    for windows in [0]:
        print('get_windows_feat user_item gap 1,2,3,4 max min {} False'.format(windows))
        if os.path.exists(path + "/sub_data/use_data/{}_uc_1_w_{}.parquet".format(name,windows)):
            t = pd.read_parquet(path + "/sub_data/use_data/{}_uc_1_w_{}.parquet".format(name,windows))
        else:
            t = get_windows_feat(data,windows,'user_category','f',[1,2,3,4],['count'],False)
            print(t.head())
            t.to_parquet(path + "/sub_data/use_data/{}_uc_1_w_{}.parquet".format(name,windows))
        base_data = pd.merge(base_data,t,on=['user_category'],how='left',copy=False)
        del t;gc.collect()

    if is_submit:
        pass
    else:
        base_data = pd.merge(base_data, label, on=['user_id', 'item_id'], how='left', copy=False)
        base_data['label'] = base_data['label'].fillna(0)
    print(base_data.shape)
    base_data = base_data.fillna(0)
#     del base_data['user_item']
#     base_data = reduce_mem_usage(base_data)
    return base_data

In [ ]:
# # 12月17日 之前的历史信息全量
train_label = pd.read_parquet("../sub_data/use_data/train_label.parquet")
train_item = pd.read_parquet("../sub_data/use_data/train_item.parquet")
user_data_list = {'2014-12-16': 0}
train_history = get_history_data(train_label,train_item,user_data_list,'for_train',True)

In [ ]:
train_label.head(),train_item.head(),train_label['label'].mean()

In [ ]:
# train_history = pd.read_parquet(path + "/sub_data/use_data/history_for_train.parquet")

In [ ]:
train_history['user_item'] = train_history[['user_id','item_id']].parallel_apply(lambda x:str(x[0]) +'_' + str(x[1]),axis=1)

In [ ]:
train_history.head()

In [ ]:
train_history.dtypes,train_label.dtypes,train_item.dtypes,

In [ ]:
# train_history = reduce_mem_usage(train_history)
# train_label = reduce_mem_usage(train_label)
# train_item = reduce_mem_usage(train_item)

In [ ]:
train_history.dtypes,train_label.dtypes,train_item.dtypes,

In [ ]:
for_train_lgb = gen_feature(train_history,train_item,train_label,False,'train_o2o')

In [ ]:
for_train_lgb.dtypes,

In [ ]:
del train_history
gc.collect()

In [ ]:
for_train_lgb['label'] = for_train_lgb['label'].astype(int)

In [ ]:
for_train_lgb['label'].mean()

In [ ]:
for_train_lgb['label'].value_counts()

In [ ]:
# # 12月18日 之前的历史信息全量
valid_label = pd.read_parquet("../sub_data/use_data/valid_label.parquet")
valid_item = pd.read_parquet("../sub_data/use_data/valid_item.parquet")
user_data_list = {'2014-12-17': 0}
valid_history = get_history_data(valid_label,valid_item,user_data_list,'for_valid',True)

In [ ]:
# valid_history = pd.read_parquet(path + "/sub_data/use_data/history_for_valid.parquet")

In [ ]:
valid_history['user_item'] = valid_history[['user_id','item_id']].parallel_apply(lambda x:str(x[0]) +'_' + str(x[1]),axis=1)

In [ ]:
# valid_history = reduce_mem_usage(valid_history)
# valid_label = reduce_mem_usage(valid_label)
# valid_item = reduce_mem_usage(valid_item)

In [ ]:
for_valid_lgb = gen_feature(valid_history,valid_item,valid_label,False,'valid_o2o')

In [ ]:
del valid_history
gc.collect()

In [ ]:
for_valid_lgb['label'] = for_valid_lgb['label'].astype(int)

In [ ]:
# for_valid_lgb.dtypes,

In [ ]:
for_valid_lgb['label'].mean()

In [ ]:
# for_valid_lgb.head()

In [ ]:
for_valid_lgb['label'].value_counts()

In [ ]:
print('train', for_train_lgb.shape, 'valid', for_valid_lgb.shape)

In [ ]:
feature = [x for x in for_train_lgb.columns if x not in ['user_category','gap' ,'label', 'user_geohash', 'predict', 'user_item','user_hour','item_hour','user_item_hour']]
label = 'label'

In [ ]:
feature

In [ ]:
lgb_train = lgb.Dataset(for_train_lgb[feature], for_train_lgb[label])

In [ ]:
lgb_eval = lgb.Dataset(for_valid_lgb[feature], for_valid_lgb[label])

In [ ]:
params = {
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'n_jobs': -1,
    'seed': 1024,
    'verbose': -1
}
callbacks = [lgb.log_evaluation(period=25), lgb.early_stopping(stopping_rounds=50)]

In [ ]:
print('Starting training...')
# feature_name and categorical_feature
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=10000,
                valid_sets=[lgb_train, lgb_eval],
                callbacks=callbacks,
                )
del lgb_train, lgb_eval
gc.collect()
# [106]	training's auc: 0.902916	valid_1's auc: 0.891666
# [107]	training's auc: 0.91216	valid_1's auc: 0.90192
# [109]	training's auc: 0.91338	valid_1's auc: 0.905031

In [ ]:
feature_imp_df = pd.DataFrame()
feature_imp_df['feature'] = gbm.feature_name()
feature_imp_df['import'] = gbm.feature_importance()

In [ ]:
feature_imp_df.sort_values('import')

In [ ]:
feature_imp_df.to_csv('./feature_imp_df.csv',index=False)

In [ ]:
for_valid_lgb['predict'] = gbm.predict(for_valid_lgb[feature], num_iteration=gbm.best_iteration).reshape(-1, 1)

In [ ]:
for_valid_lgb['predict']

In [ ]:
for_valid_label = pd.read_parquet("../sub_data/use_data/valid_label.parquet")
for_valid_label['user_item'] = for_valid_label[['user_id','item_id']].parallel_apply(lambda x:str(x[0]) +'_' + str(x[1]),axis=1)
for_valid_label_set = set(for_valid_label['user_item'])

In [ ]:
for_valid_ = for_valid_lgb.groupby(['user_id','item_id'])['predict'].mean().reset_index()
for_valid_.columns = ['user_id','item_id','predict']

In [ ]:
for_valid_.shape

In [ ]:
t = for_valid_.sort_values(['predict'])[-320000:]
print(t.shape)

In [ ]:
t['user_item'] = t[['user_id','item_id']].parallel_apply(lambda x:str(x[0]) +'_' + str(x[1]),axis=1)

In [ ]:
t_and_for_valid_label = set(t['user_item']) & for_valid_label_set
p = len(t_and_for_valid_label) / len(set(t['user_item']))
r = len(t_and_for_valid_label) / len(for_valid_label_set)

In [ ]:
print((2 * p * r) / (p + r),p,r )
# 0.07642381738543827 0.09337428614367298 0.06468194195477274
# F1:0.0614  Recall:0.0528 Precision:0.0733
# 0.07542477247013826 0.0980864981979542 0.061269239152712755
# 0.07776773603280941 0.076234375 0.07936404657412137

In [ ]:
# # 12月19日 之前的历史信息全量
submit_item = pd.read_csv('../data/part1_item.txt', header=None, sep='\t')
submit_item.columns = ['item_id', 'item_geohash', 'item_category']
user_data_list = {'2014-12-18': 0}
submit_history =get_history_data('',submit_item,user_data_list,'for_submit',False)

In [ ]:
# submit_history = pd.read_parquet(path + "/sub_data/use_data/history_for_submit.parquet")

In [ ]:
submit_history['user_item'] = submit_history[['user_id','item_id']].parallel_apply(lambda x:str(x[0]) +'_' + str(x[1]),axis=1)

In [ ]:
# submit_history = reduce_mem_usage(submit_history)
# submit_item = reduce_mem_usage(submit_item)

In [ ]:
del submit_item['item_geohash']
gc.collect()

In [ ]:
submit_item.dtypes,submit_history.dtypes

In [ ]:
gc.collect()

In [ ]:
for_submit_lgb = gen_feature(submit_history,submit_item,'',True,'submit_o2o')

In [ ]:
del submit_history
gc.collect()

In [ ]:
# del for_train_lgb;gc.collect()

In [ ]:
# for_train_valid_lgb = for_valid_lgb

In [ ]:
for_train_valid_lgb = pd.concat([for_train_lgb,for_valid_lgb],copy=False)

In [ ]:
del for_train_lgb,for_valid_lgb;gc.collect()

In [ ]:
lgb_train_valid = lgb.Dataset(for_train_valid_lgb[feature], for_train_valid_lgb[label])

In [ ]:
del for_train_valid_lgb

In [ ]:
gc.collect()

In [ ]:
print('Starting training...')
# feature_name and categorical_feature
gbm2 = lgb.train(params,
                lgb_train_valid,
                num_boost_round=int(gbm.best_iteration * 1.1),
                valid_sets=[lgb_train_valid, lgb_train_valid],
                callbacks=callbacks,
                )

In [ ]:
# 提交结果
for_submit_lgb['predict'] = gbm2.predict(for_submit_lgb[feature], num_iteration=gbm.best_iteration).reshape(-1, 1)
print(for_submit_lgb.shape)

In [ ]:
submit_ = for_submit_lgb.groupby(['user_id','item_id'])['predict'].mean().reset_index()
submit_.columns = ['user_id','item_id','predict']

In [ ]:
submit = submit_.sort_values('predict')

In [ ]:
submit = submit[['user_id','item_id','predict']][-35000:]
print(submit.shape)

In [ ]:
submit[submit['item_id'].isin(list(submit_item['item_id'].unique()))]

In [ ]:
# submit[['user_id','item_id']].to_csv(path + '/submit_35000_blind_1_1.txt', sep='\t', index=False, header=None, encoding='utf-8')

In [ ]:
import datetime
submit[['user_id','item_id']].to_csv(("../submit/submit_"+datetime.datetime.now().strftime('%Y%m%d_%H%M%S') + ".csv"), header=None, index=False)